# Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
import numpy.matlib
import pandas as pd
import time
import os
from os import listdir
from os.path import join, isfile

!pip install jupyter-dash
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import requests
import io
from plotly.subplots import make_subplots

!pip install dash-bootstrap-components
import dash
import dash_bootstrap_components as dbc

# Data Processing

In [ ]:
path = 'https://github.com/BobbyWilt/PD-Watch-Dashboard/raw/main/Data/data/110001/'

In [ ]:
# test.drop(columns=['Unnamed: 0'], inplace=True)
cols = ['left_acc_x', 'left_acc_y', 'left_acc_z',
                'left_gyr_x', 'left_gyr_y', 'left_gyr_z',
                'right_acc_x', 'right_acc_y', 'right_acc_z',
                'right_gyr_x', 'right_gyr_y', 'right_gyr_z',
                'chest_acc_x', 'chest_acc_y', 'chest_acc_z',
                'chest_gyr_x', 'chest_gyr_y', 'chest_gyr_z'
               ]

In [ ]:
# Loop through files and load csv's into dataframe.  Reset columns and index
for i in range(1, 8):
    url = path + f'data_segment_{i}.csv'
    locals()[f'df{i}'] = pd.read_csv(url)
    locals()[f'df{i}'].set_index('time', inplace=True)
    locals()[f'df{i}'].columns = cols

# Figures

In [ ]:
# Make loop to make all time-series plots
for i in range(1,8):
    # Mulit-line plot - separated by sensor
    locals()[f'fig{i}'] = make_subplots(rows=6, cols=1, shared_xaxes=True,
                                        vertical_spacing=0.06,
                                        subplot_titles=[
                                            'Accelerometry Left', 'Gyro Left',
                                            'Accelerometry Right', 'Gyro Right',
                                            'Accelerometry Chest', 'Gyro Chest'])
    data = locals()[f'df{i}']
    sensor_axis_types = ['acc_x', 'acc_y', 'acc_z',
                         'gyr_x', 'gyr_y', 'gyr_z']

    count, count_l = 1, 0
    # Loop thorugh each sensor
    for sens in ['left', 'right', 'chest']:
    # Loop through each sensor data type
        for d_type in ['acc', 'gyr']:
            # loop through each sensor axis
            for axis, color in zip(['x', 'y', 'z'],
                                   ['MediumSeaGreen', 'SandyBrown', 'DodgerBlue']):
                count_l+=1
                if count_l > 3:
                    legend_vis=False
                else:
                    legend_vis=True

                locals()[f'fig{i}'].add_trace(
                    go.Scatter(x=data.index, y=data[f'{sens}_{d_type}_{axis}'],
                               marker_color=color, name=f'{axis}-axis', showlegend=legend_vis),
                    row=count, col=1)
            count += 1

    locals()[f'fig{i}'].update_layout(height=1500, width=1500, title_text=f"Data Segment {i}")

In [ ]:
quest_url = 'https://raw.githubusercontent.com/BobbyWilt/PD-Watch-Dashboard/main/Data/data/110001/110001_features900.csv'
df_quest = pd.read_csv(quest_url)
df_quest.head()

In [ ]:
# Patient questionairre questions
mood_cats = ['mood_well', 'mood_down', 'mood_fright', 'mood_tense',
             'phy_sleepy', 'phy_tired', 'mood_cheerf', 'mood_relax']
mood_nams = ['Well', 'Down', 'Frightened', 'Tense', 'Sleepy', 'Tired',
             'Cheerful', 'Relaxed']
symptom_cats = ['tremor', 'slowness', 'stiffness', 'muscle_tension', 'dyskinesia']
functionality_cats = ['act_problemless', 'mobility_well', 'sit_still',
                      'speech_well', 'walk_well']
functionality_nams = ['Current Activity', 'Mobility Comfort', 'Sit & Stand',
                      'Speech', 'Walk']

In [ ]:
# Mood Scatterpolar
df_mood = df_quest[mood_cats]
fig=go.Figure()
fig.add_trace(
    go.Scatterpolar(name='mood', r =df_mood.iloc[2], theta = mood_nams,fill='toself',
          opacity=0.75 )
)
fig.update_layout(title='Mood')
fig.update_polars(radialaxis_range=(1,7))
fig.show()

In [ ]:
# Symptoms - scatter plar
df_sympt = df_quest[symptom_cats]
fig=go.Figure()
fig.add_trace(
    go.Scatterpolar(r =df_sympt.iloc[1], theta = symptom_cats, fill='toself',
          opacity=0.75 )
)
fig.update_layout(title='Symptoms')
fig.update_polars(radialaxis_range=(1,7))
fig.show()

In [ ]:
# Functionality - scatter polar
df_func = df_quest[functionality_cats]
fig=go.Figure()
fig.add_trace(
    go.Scatterpolar(r =df_func.iloc[1], theta=functionality_nams, fill='toself',
          opacity=0.75 )
)
fig.update_layout(title='Ease of Functionality')
fig.update_polars(radialaxis_range=(1,7))
fig.show()

In [ ]:
df_func = df_quest[functionality_cats]
fig2=go.Figure()
fig2.add_trace(
    go.Scatterpolar(r =df_func.iloc[1], theta=functionality_nams, fill='toself',
          opacity=0.75 )
)
fig2.update_layout(title='Ease of Functionality')
fig2.update_polars(radialaxis_range=(1,7))
fig2.show()

# Dashboard

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Div('Data Segment Selector'),
            dcc.RadioItems([1 , 2, 3, 4, 5, 6, 7],1, id='data_segment_index'),
            dcc.Graph(id='time-series')
        ], width=9, lg=4.5
        ),
        dbc.Col([
            dcc.Graph(id='sp_mood'),
            dcc.Graph(id='sp_symptom'),
            dcc.Graph(id='sp_func')
        ], width=3, lg=1.5
        )
    ], align='start')
], fluid=True)

@app.callback(
    Output('time-series', 'figure'),
    Input('data_segment_index', 'value')
)
def select_time_figure(data_segment_index):
    figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7]
    fig_ts = figures[data_segment_index-1]
    return fig_ts


@app.callback(
    Output('sp_mood', 'figure'),
    Input('data_segment_index', 'value')
)
def create_sp_mood(data_segment_index):
    fig_sp_m = go.Figure()
    fig_sp_m.add_trace(
        go.Scatterpolar(name='Mood', r=df_mood.iloc[data_segment_index-1],
                        theta=mood_cats,fill='toself', opacity=0.75 )
    )
    fig_sp_m.update_layout(title='Mood')
    fig_sp_m.update_polars(radialaxis_range=(1,7))
    return fig_sp_m

@app.callback(
    Output('sp_symptom', 'figure'),
    Input('data_segment_index', 'value')
)
def create_sp_sympt(data_segment_index):
    fig_sp_s=go.Figure()
    fig_sp_s.add_trace(
        go.Scatterpolar(r =df_sympt.iloc[data_segment_index-1],
                        theta=symptom_cats, fill='toself', opacity=0.75 )
    )
    fig_sp_s.update_layout(title='Symptoms')
    fig_sp_s.update_polars(radialaxis_range=(1,7))
    return fig_sp_s

@app.callback(
    Output('sp_func', 'figure'),
    Input('data_segment_index', 'value')
)
def create_sp_f(data_segment_index):
    fig=go.Figure()
    fig.add_trace(
        go.Scatterpolar(r =df_func.iloc[data_segment_index-1],
                        theta=functionality_nams, fill='toself', opacity=0.75 )
    )
    fig.update_layout(title='Ease of Functionality')
    fig.update_polars(radialaxis_range=(1,7))
    return fig
print('Click on the app link below to view the dashboard.')
app.run_server()